# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [4]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
from sympy import solve_poly_system
from sympy.abc import x, y, z

# Provided functions

In [5]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1
    return n, lines, centers

In [6]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [7]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [8]:
#Takes the input as number of vanishing lines, coordinates of the vanishing lines and their centers
def get_vanishing_point(n, lines, centers):
    """
    Solves for the vanishing point using the user-input lines.
    """
    homo_points =np.zeros((int(n*(n-1)/2), 3)) #Points in 3D homogeneous coordinates
    count = 0
    #Taking the intersection of all vanishing lines 2 at a time and calculating the intersection points
    for i in range(n):
        for j in range(i+1, n):
            homo_points[count,:] = np.cross(lines[:, i], lines[:, j])
            count+=1
    
    #Points in 3D homogeneous coordinates with last entry normalized as 1
    points = np.array([homo_points[i]/homo_points[i][-1] for i in range(homo_points.shape[0])][:2])
    
    #Averaging all the intersection points to get a vanishing point where approximately all the vanishing lines intersect
    vans_point = points.mean(axis = 0)
    print ('Vanishing Point = ', vans_point)
    return vans_point

In [9]:
#Takes the input as the calculated vanishing points 
def get_horizon_line(vansh_points):
    """
    Calculates the ground horizon line.
    """
    horizon_line = np.cross(vansh_points[0], vansh_points[1]) #Intersection line of the two horizontal vanishing points 
    return horizon_line

In [23]:
#Takes the input as the image and the vanishing points' coordinates
def plot_horizon_line(im, vansh_points):
    """
    Plots the horizon line.
    """
    #Taking the 2 horizontal vanishing points to draw a horizon line
    plt.figure()
    plt.imshow(im)
    plt.plot([vansh_points[0][0], vansh_points[0][1]], [vansh_points[1][0], vansh_points[1][1]], 'g')
    plt.plot(vansh_points[0][0], vansh_points[1][0], 'ro')
    plt.plot(vansh_points[0][1], vansh_points[1][1], 'ro')
    plt.savefig('horizon.jpg')
    plt.show()

In [11]:
#Takes the input as the vanishing points' coordinates
def get_camera_parameters(vansh_points):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    from sympy.abc import x, y, z
    w = np.array([[z**2, 0, -x*(z**2)], [0, z**2, -y*(z**2)], [-x*(z**2), -y*(z**2), (x**2)*(z**2) + (y**2)*(z**2) + 1 ]]) 
    #Using orthogonal points, w = K^(-T)*K^(-1)
    poly1 = np.dot(np.dot(vansh_points[:, 0], w), vansh_points[:, 1])
    poly2 = np.dot(np.dot(vansh_points[:, 0], w), vansh_points[:, 2])
    poly3 = np.dot(np.dot(vansh_points[:, 1], w), vansh_points[:, 2])
    print (poly1, poly2, poly3)
    solutions = solve_poly_system([poly1, poly2, poly3], x, y, z) #Solving the polynomial system with 3 polynomials 
    print (solutions)
    #Using u, v must be in image bounds and f>0
    if solutions[0][2]>0:
        solution = solutions[0]
    else:
        solution = solutions[1]
    u = solution[0]
    v = solution[1]
    f = 1./solution[2]
    print ('u0 = ', u, 'v0 = ', v, 'f = ', f)   
    K = np.array([[f, 0., u], [0, f, v], [0, 0, 1]]) #Forming the K matrix using three points are orthogonal 
    return f, u, v

In [60]:
#Takes the input as vanishing points, focal length, v and v (coordinates for the principal point)
def get_rotation_matrix(vansh_points, focal, u1, v1):
    """
    Computes the rotation matrix using the camera parameters.
    """
    K = np.array([[focal, 0., u1], [0, focal, v1], [0, 0, 1]]) #Forming the K matrix using three points are orthogonal 
    #Right vanishing point = x-direction
    #Left vansihing point = z-direction
    #Vertical vanishing point = y-direction
    
    K =(K.astype('float'))
    #Solving for lambda_i 
    lambda1 = 1./np.linalg.norm(np.dot(np.linalg.inv(K), vansh_points[:,1])) #Right vanishing point = x-direction
    lambda2 = 1./np.linalg.norm(np.dot(np.linalg.inv(K), vansh_points[:,2])) #Vertical vanishing point = y-direction
    lambda3 = 1./np.linalg.norm(np.dot(np.linalg.inv(K), vansh_points[:,0])) #Left vansihing point = z-direction
    R1 = lambda1*np.dot(np.linalg.inv(K), vansh_points[:,1]) #Right vanishing point = x-direction
    R2 = lambda2*np.dot(np.linalg.inv(K), vansh_points[:,2]) #Vertical vanishing point = y-direction
    R3 = lambda3*np.dot(np.linalg.inv(K), vansh_points[:,0]) #Left vansihing point = z-direction
    R = np.vstack((np.vstack((R1, R2)), R3))
    rotation_matrix = R.T
    print ('Rotation Matrix = ', rotation_matrix)
    return rotation_matrix

In [62]:
#Takes the input as vanishing points, the coordinates of the person and the object (for which we need to estimate the height)
def estimate_height(vansh_points, coords_person, coords_object, person_height):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    b = coords_object[:,1]
    r = coords_object[:,0]
    b0 = coords_person[:, 1]
    t0 = coords_person[:, 0]
    vx = vansh_points[:,0]
    vy = vansh_points[:,1]
    vz = vansh_points[:,2]
    v = np.cross(np.cross(b, b0), np.cross(vx, vy))
    v_normalized = v/v[2]
    print ('vx =', vx, 'v = ', v_normalized, 'vz = ', vz)
    t = np.cross(np.cross(v_normalized, t0), np.cross(r, b))
    t_normalized = t/t[2]
    print ('b =', b, 'r=', r, 't=', t_normalized)
    #Computing the cross ratio
    ratio = (np.linalg.norm(t_normalized-b)*np.linalg.norm(vz-r))/(np.linalg.norm(r-b)*np.linalg.norm(vz-t_normalized)) 
    print ('cross ratio = ', ratio)
    object_height = person_height/ratio
    #Returns object height in inches
    return object_height

# Main function

In [47]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(n, lines, centers) #Giving vanishing lines as input
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts) #Giving vanishing points as input
print ('Horizon line', horizon_line)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Vanishing Point =  [ 36.91624027 208.88281338   1.        ]
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Vanishing Point =  [1.35959484e+03 2.28755114e+02 1.00000000e+00]
Getting vanish

In [30]:
#Plot the ground horizon line
plot_horizon_line(im, vpts) #Giving image and vanishing point coordinates as input

In [48]:
#Normalizing the horizon line by sqrt(a^2+b^2) such that a^2+b^2 = 1
horizon_line_normalized = horizon_line/np.sqrt(horizon_line[0]**2+horizon_line[1]**2)
print ('Coordinates of normalized horizon line ', horizon_line_normalized)

Coordinates of normalized horizon line  [ 0.99984348 -0.01769199 -0.02448773]


In [49]:
# Part 2
#Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts) #Giving vanishing points as input

1.0*x**2*z**2 - 1396.51108449084*x*z**2 + 1.0*y**2*z**2 - 437.637927802021*y*z**2 + 97974.1418106742*z**2 + 1.0 1.0*x**2*z**2 - 561.935107476177*x*z**2 + 1.0*y**2*z**2 - 8572.37755289902*y*z**2 + 1766372.03354564*z**2 + 1.0 1.0*x**2*z**2 - 1884.61371143309*x*z**2 + 1.0*y**2*z**2 - 8592.24985393626*y*z**2 + 2627005.14106401*z**2 + 1.0
[(646.596322951620, 271.432369253008, -0.00152139598687154), (646.596322951620, 271.432369253008, 0.00152139598687154)]
u0 =  646.596322951620 v0 =  271.432369253008 f =  657.291072560478


In [61]:
# Part 3
# Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, u, v) #Giving vanishing points and f, u, v as input

#Check that R is orthogonal matrix - last row may be all 1s

Rotation Matrix =  [[ 0.73453504 -0.01497329 -0.67840554]
 [-0.04396635  0.99660561 -0.06960038]
 [ 0.67714492  0.08095093  0.73138342]]


In [52]:
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts


In [63]:
#Estimate heights
person_heights = [5.5, 6] # 5feet 6 inches = 5.5 feet
for person_height in person_heights:
    print ('Person Height = ', person_height)
    for obj in objects[1:]:
        print('Estimating height of %s' % obj)
        height = estimate_height(vpts, coords['person'], coords[obj], person_height) 
        #Giving vanishing points, coordinates of person and object as input
        print ('Height of', obj, ' = ', height, 'feet')

Person Height =  5.5


NameError: name 'objects' is not defined